# Arnold Tongue

In [3]:
import zipfile
import numpy as np
import os
# Path to the zipped 2D embedded data
zip_file_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Phase Space/2dembedded_data.zip'
extraction_dir = '/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Phase Space/temp_extraction'

# Extract zipped data to a temporary directory
with zipfile.ZipFile(zip_file_path, 'r') as zipf:
    zipf.extractall(extraction_dir)

eeg_channels = ['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6',
                'M1', 'T7', 'C3', 'Cz', 'C4', 'T8', 'M2', 'CP5', 'CP1', 'CP2', 'CP6',
                'P7', 'P3', 'Pz', 'P4', 'P8', 'POz', 'O1', 'Oz', 'O2']


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def kuramoto_model(theta, t, omega, K, N, y):
    """ Kuramoto model to describe synchronization phenomena.
    y is the driving force with amplitude a and frequency b.
    """
    dydt = omega + (K/N) * np.sum(np.sin(theta - theta[:, np.newaxis]), axis=1) + y(t)
    return dydt

def driving_force(t, a, b):
    """ Periodic driving force with amplitude 'a' and frequency 'b'. """
    return a * np.cos(2 * np.pi * b * t)

# Parameters
N = len(eeg_channels) # Number of oscillators (one for each EEG channel)
K = 1.0 # Coupling strength
omega = np.random.uniform(-1, 1, N) # Natural frequencies

# Ranges for amplitude 'a' and frequency 'b' of the driving force
a_values = np.linspace(0.1, 2.0, 50)
b_values = np.linspace(0.1, 2.0, 50)

# Store results
synchronization_array = np.zeros((len(a_values), len(b_values)))

# Iterate over values of a and b
for i, a in enumerate(a_values):
    for j, b in enumerate(b_values):
        y = lambda t: driving_force(t, a, b)
        
        # Integrate the Kuramoto model with the given driving force
        # We'll use the odeint function from scipy for this purpose
        from scipy.integrate import odeint
        theta_0 = np.random.uniform(0, 2 * np.pi, N)
        T = np.linspace(0, 100, 1000)
        solution = odeint(kuramoto_model, theta_0, T, args=(omega, K, N, y))
        
        # Measure synchronization
        r = np.abs(np.sum(np.exp(1j * solution), axis=1)) / N
        synchronization_array[i, j] = np.mean(r)

# Plot
plt.imshow(synchronization_array, extent=[a_values.min(), a_values.max(), b_values.min(), b_values.max()], origin='lower', aspect='auto')
plt.colorbar(label="Synchronization")
plt.xlabel('Amplitude a')
plt.ylabel('Frequency b')
plt.title('Arnold Tongues')
plt.show()
